###  importing relevant libraries

In [1]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
from collections import Counter
from tabulate import tabulate
from tqdm import tqdm
import seaborn as sns
import pandas as pd
import numpy as np
%matplotlib inline
import warnings
import sklearn
import time
import os
warnings.filterwarnings("ignore")

### Discovering Labels

In [2]:
def target_name(name):
    df = pd.read_csv(name,usecols=["Label"])
    target_names=sorted(list(df["Label"].unique()))
    return target_names

In [3]:
def folder(f_name): #this function creates a folder.
    try:
        if not os.path.exists(f_name):
            os.makedirs(f_name)
    except OSError:
        print ("Tthe folder could not be created!")

In [4]:
def find_the_way(path,file_format,con=""):
    files_add = []
    # r=root, d=directories, f = files
    for r, d, f in os.walk(path):
        for file in f:
            if file_format in file:
                if con in file:
                    files_add.append(os.path.join(r, file))  
            
    return files_add

In [5]:
altime=0
#def most_frequent(List): 
#    return max(set(List), key = List.count) 


 
def most_frequent(List):
    occurence_count = Counter(List)
    occurence_count={k: v for k, v in sorted(occurence_count.items(), key=lambda item: item[1],reverse=True)}
    big=list(occurence_count.values())
    big=big.count(big[0])
    return list(occurence_count.keys())[np.random.randint(big)]


def split(a, n):
    k, m = divmod(len(a), n)
    return (a[i * k + min(i, m):(i + 1) * k + min(i + 1, m)] for i in range(n))
def create_exception(df): 
    exception_list=[]
    dominant_mac=[]
    for i in df['aggregated'].unique():
        k=df[df['aggregated']==i]
        for ii in ['MAC']:
            hist = {}
            for x in k[ii].values:
                hist[x] = hist.get(x, 0) + 1
            hist=dict(sorted(hist.items(), key=lambda item: item[1],reverse=True))
            temp=next(iter(hist))
            if temp not in dominant_mac:
                dominant_mac.append(temp)
            else:
                exception_list.append(temp)
    return exception_list




def merged(m_test,predict,step,mixed):
    second=time.time()
    mac_test=[]
    for q in m_test.index:
        mac_test.append(m_test[q])

    d_list=sorted(list(m_test.unique()))
    devices={}
    for q in d_list:
        devices[q]=[]    


    new_y=[0]*len(m_test)

    for q,qq in enumerate (mac_test):
        devices[qq].append(q)
    for q in devices:
        a = [devices[q][j:j + step] for j in range(0, len(devices[q]), step)]  
        for qq in a:
            step_list=[]
            for qqq in qq:
                step_list.append(predict[qqq])
            add=most_frequent(list(step_list))
            for qqq in qq:
                new_y[qqq]=add
    results=pd.DataFrame(m_test)
    results["aggregated"]=new_y
    results["normal"]=predict
    
    #MIXED METHOD
    if mixed:
        exception=create_exception(results)
        for q in exception:
            results.loc[results.MAC == q, 'aggregated'] = results['normal']

    return results["aggregated"].values,time.time()-second

## Calculation of evaluations

In [6]:
def score(altime,train_time,test_time,predict,y_test,class_based_results,i,cv,dname,ml):
    rc=sklearn.metrics.recall_score(y_test, predict,average= "macro")
    pr=sklearn.metrics.precision_score(y_test, predict,average= "macro")
    f_1=sklearn.metrics.f1_score(y_test, predict,average= "macro")     
    accuracy=sklearn.metrics.accuracy_score(y_test, predict)
    accuracy_b=sklearn.metrics.balanced_accuracy_score( y_test,predict)
    kappa=sklearn.metrics.cohen_kappa_score(y_test, predict,labels=None, weights=None, sample_weight=None)
    try:
        roc=sklearn.metrics.roc_auc_score(y_test, predict)
    except:roc=0
    report = sklearn.metrics.classification_report(y_test, predict, target_names=target_names,output_dict=True)
    cr = pd.DataFrame(report).transpose()
    line=[dname,i,cv,ml,accuracy,accuracy_b,pr,rc,f_1,kappa,roc,train_time,test_time,altime]

    if class_based_results.empty:
        class_based_results =cr
    else:
        class_based_results = class_based_results.add(cr, fill_value=0)
    return line,class_based_results

In [7]:
def ML(loop1,loop2,output_csv1,cols,step,mixed,dname):
    from sklearn.metrics import balanced_accuracy_score
    from sklearn.preprocessing import Normalizer
    
    for ii in ml_list:
        class_based_results=pd.DataFrame()#"" #pd.DataFrame(0, index=np.arange((len(target_names)+3)), columns=["f1-score","precision","recall","support"])
        cm=pd.DataFrame()
        cv=0
        output_csv=output_csv1.replace("ML",ii)
        repetition=5
        lines=[["Dataset","T","CV","ML","Acc","b_Acc","Prec","Rec","F1","kap","ROC","tra-T","test-T","Al-Time"]]
        for i in range(repetition):



            #TRAIN
            df = pd.read_csv(loop1,usecols=cols)
            m_train=df["MAC"]
            del df["MAC"]
            X_train =df[df.columns[0:-1]]
            X_train=np.array(X_train)
            df[df.columns[-1]] = df[df.columns[-1]].astype('category')
            y_train=df[df.columns[-1]].cat.codes  

            #TEST
            df = pd.read_csv(loop2,usecols=cols)
            df = shuffle(df, random_state=42)
            m_test=df["MAC"]
            del df["MAC"]
            X_test =df[df.columns[0:-1]]
            X_test=np.array(X_test)
            df[df.columns[-1]] = df[df.columns[-1]].astype('category')
            y_test=df[df.columns[-1]].cat.codes




            results_y=[]
            cv+=1
            results_y.append(y_test)


            #machine learning algorithm is applied in this section
            clf = ml_list[ii]#choose algorithm from ml_list dictionary
            second=time.time()
            clf.fit(X_train, y_train)
            train_time=(float((time.time()-second)) )
            second=time.time()
            predict =clf.predict(X_test)
            test_time=(float((time.time()-second)) )
            if step==1:
                altime=0
                line,class_based_results=score(altime,train_time,test_time,predict,y_test,class_based_results,i,cv,dname,ii)
            else:
                predict,altime=merged(m_test,predict,step,mixed)
                line,class_based_results=score(altime,train_time,test_time,predict,y_test,class_based_results,i,cv,dname,ii)
                
                
            lines.append(line)
            df_cm = pd.DataFrame(sklearn.metrics.confusion_matrix(y_test, predict))
            if cm.empty:
                cm =df_cm
            else:
                cm = cm.add(df_cm, fill_value=0)
            
        class_based_results=class_based_results/(repetition)
        results = pd.DataFrame (lines[1:], columns = lines[0])
        results.to_csv(output_csv.replace("ML",ii),index=False)
        results=results.round(3)
        print (tabulate(results, headers=list(results.columns)))
        print()
  
        
        #print (tabulate(class_based_results, headers=list(class_based_results.columns)))
        class_based_results.to_csv(output_csv.replace(".csv","CB.csv"))
        if True:
            cm=cm//repetition
            graph_name=output_csv+"_confusion matrix.pdf"   
            plt.figure(figsize = (40,28))
            sns.heatmap(cm,xticklabels=target_names, yticklabels=target_names, annot=True, fmt='g')
            plt.savefig(graph_name,bbox_inches='tight')#, dpi=400)
            cm.to_csv(graph_name.replace(".pdf",".csv"))
            #plt.show()
            #print(cm)
            #print("\n\n\n")             


            plt.clf()

            
            
            
            
            



# Machine learning applications 

# Aalto Dataset

In [8]:
feature= ['pck_size', 'Ether_type', 'LLC_ctrl', 'EAPOL_version', 'EAPOL_type', 'IP_ihl', 'IP_tos', 'IP_len', 'IP_flags', 'IP_DF', 'IP_ttl', 'IP_options', 'ICMP_code', 'TCP_dataofs', 'TCP_FIN', 'TCP_ACK', 'TCP_window', 'UDP_len', 'DHCP_options', 'BOOTP_hlen', 'BOOTP_flags', 'BOOTP_sname', 'BOOTP_file', 'BOOTP_options', 'DNS_qr', 'DNS_rd', 'DNS_qdcount', 'dport_class', 'payload_bytes', 'entropy',
"MAC",
'Label']


In [9]:
ml_list={"DT" :DecisionTreeClassifier()}



In [10]:
datasets=[['./csvs\\A211102.csv',
  './csvs\\A211105.csv',
  './csvs\\I211102.csv',
  './csvs\\I211103.csv',
  './csvs\\I211104.csv'],
 ['./csvs\\A211108.csv',
  './csvs\\A211109.csv',
  './csvs\\A211115.csv',
  './csvs\\A211116.csv',
  './csvs\\A211117.csv',
  './csvs\\A211118.csv',
  './csvs\\A211122.csv',
  './csvs\\A211123.csv',
  './csvs\\A211125.csv',
  './csvs\\A211206.csv',
  './csvs\\A211208.csv',
  './csvs\\A211223.csv',
  './csvs\\A211225.csv',
  './csvs\\A211228.csv',
  './csvs\\I211108.csv',
  './csvs\\I211112.csv',
  './csvs\\I211115.csv',
  './csvs\\I211118.csv',
  './csvs\\I211119.csv',
  './csvs\\I211122.csv'],
 ['./csvs\\A211110.csv', './csvs\\A211112.csv', './csvs\\I211110.csv'],
 ['./csvs\\A211124.csv',
  './csvs\\I211124.csv',
  './csvs\\I211125.csv',
  './csvs\\I211201.csv',
  './csvs\\I211202.csv',
  './csvs\\I211203.csv',
  './csvs\\I211207.csv',
  './csvs\\I211208.csv',
  './csvs\\I211209.csv',
  './csvs\\I211210.csv',
  './csvs\\I211213.csv',
  './csvs\\I211215.csv',
  './csvs\\I211216.csv',
  './csvs\\I211220.csv',
  './csvs\\I211222.csv',
  './csvs\\I211223.csv'],
 ['./csvs\\A211126.csv', './csvs\\A211207.csv'],
 ['./csvs\\I211116.csv', './csvs\\I211117.csv']]


In [11]:
datasets=[['./csvs\\A211102.csv',
  './csvs\\A211105.csv',
  './csvs\\I211102.csv',
  './csvs\\I211103.csv',
  './csvs\\I211104.csv']]


In [12]:
say=0
dataset="./results/vs/"
folder(dataset)
for no, i in enumerate(datasets):
    for ii in i:
        for iii in tqdm(i):
            say+=1
            train=ii
            test=iii
            mixed=False
            step=1
            sayac=1
            dataset="./results/vs/"
            output_csv=f"{dataset}/@{ii[7:-4]}@{iii[7:-4]}.csv"
            dataset=f"{ii[7:-4]}-{iii[7:-4]}"
            target_names=target_name(test)
            ML(train,test,output_csv,feature,step,mixed,dataset+"_"+str(step))   


  0%|                                                                                            | 0/5 [00:00<?, ?it/s]

    Dataset              T    CV  ML      Acc    b_Acc    Prec    Rec     F1    kap    ROC    tra-T    test-T    Al-Time
--  -----------------  ---  ----  ----  -----  -------  ------  -----  -----  -----  -----  -------  --------  ---------
 0  A211102-A211102_1    0     1  DT    0.934    0.905   0.919  0.905  0.909  0.931      0    1.462     0.158          0
 1  A211102-A211102_1    1     2  DT    0.934    0.905   0.919  0.905  0.909  0.931      0    1.454     0.156          0
 2  A211102-A211102_1    2     3  DT    0.934    0.905   0.919  0.905  0.909  0.931      0    1.545     0.15           0
 3  A211102-A211102_1    3     4  DT    0.934    0.905   0.919  0.905  0.909  0.931      0    1.384     0.144          0
 4  A211102-A211102_1    4     5  DT    0.934    0.905   0.919  0.905  0.909  0.931      0    1.299     0.157          0



 20%|████████████████▊                                                                   | 1/5 [00:36<02:24, 36.07s/it]

    Dataset              T    CV  ML      Acc    b_Acc    Prec    Rec     F1    kap    ROC    tra-T    test-T    Al-Time
--  -----------------  ---  ----  ----  -----  -------  ------  -----  -----  -----  -----  -------  --------  ---------
 0  A211102-A211105_1    0     1  DT    0.889    0.823   0.856  0.823  0.826  0.884      0    1.409     0.156          0
 1  A211102-A211105_1    1     2  DT    0.889    0.823   0.848  0.823  0.824  0.884      0    1.237     0.111          0
 2  A211102-A211105_1    2     3  DT    0.889    0.823   0.847  0.823  0.824  0.884      0    1.308     0.158          0
 3  A211102-A211105_1    3     4  DT    0.889    0.823   0.848  0.823  0.824  0.884      0    1.244     0.145          0
 4  A211102-A211105_1    4     5  DT    0.889    0.823   0.856  0.823  0.827  0.884      0    1.544     0.201          0



 40%|█████████████████████████████████▌                                                  | 2/5 [01:10<01:46, 35.38s/it]

...

<Figure size 4000x2800 with 0 Axes>

<Figure size 4000x2800 with 0 Axes>

<Figure size 4000x2800 with 0 Axes>

<Figure size 4000x2800 with 0 Axes>

<Figure size 4000x2800 with 0 Axes>

In [13]:
datasets=[
 ['./csvs\\A211108.csv',
  './csvs\\A211109.csv',
  './csvs\\A211115.csv',
  './csvs\\A211116.csv',
  './csvs\\A211117.csv',
  './csvs\\A211118.csv',
  './csvs\\A211122.csv',
  './csvs\\A211123.csv',
  './csvs\\A211125.csv',
  './csvs\\A211206.csv',
  './csvs\\A211208.csv',
  './csvs\\A211223.csv',
  './csvs\\A211225.csv',
  './csvs\\A211228.csv',
  './csvs\\I211108.csv',
  './csvs\\I211112.csv',
  './csvs\\I211115.csv',
  './csvs\\I211118.csv',
  './csvs\\I211119.csv',
  './csvs\\I211122.csv']]


In [14]:
say=0
dataset="./results/vs/"
folder(dataset)
for no, i in enumerate(datasets):
    for ii in i:
        for iii in tqdm(i):
            say+=1
            train=ii
            test=iii
            mixed=False
            step=1
            sayac=1
            dataset="./results/vs/"
            output_csv=f"{dataset}/@{ii[7:-4]}@{iii[7:-4]}.csv"
            dataset=f"{ii[7:-4]}-{iii[7:-4]}"
            target_names=target_name(test)
            ML(train,test,output_csv,feature,step,mixed,dataset+"_"+str(step))   


  0%|                                                                                           | 0/20 [00:00<?, ?it/s]

    Dataset              T    CV  ML      Acc    b_Acc    Prec    Rec     F1    kap    ROC    tra-T    test-T    Al-Time
--  -----------------  ---  ----  ----  -----  -------  ------  -----  -----  -----  -----  -------  --------  ---------
 0  A211108-A211108_1    0     1  DT    0.946    0.887   0.909  0.887  0.887  0.941      0    2.188     0.219          0
 1  A211108-A211108_1    1     2  DT    0.946    0.887   0.909  0.887  0.887  0.941      0    2.125     0.219          0
 2  A211108-A211108_1    2     3  DT    0.946    0.887   0.909  0.887  0.887  0.941      0    2.141     0.219          0
 3  A211108-A211108_1    3     4  DT    0.946    0.887   0.909  0.887  0.887  0.941      0    2.125     0.219          0
 4  A211108-A211108_1    4     5  DT    0.946    0.887   0.909  0.887  0.887  0.941      0    2.219     0.219          0



  5%|████▏                                                                              | 1/20 [00:48<15:12, 48.05s/it]

    Dataset              T    CV  ML      Acc    b_Acc    Prec    Rec     F1    kap    ROC    tra-T    test-T    Al-Time
--  -----------------  ---  ----  ----  -----  -------  ------  -----  -----  -----  -----  -------  --------  ---------
 0  A211108-A211109_1    0     1  DT    0.906    0.851   0.859  0.851  0.848  0.901      0    2.078     0.188          0
 1  A211108-A211109_1    1     2  DT    0.906    0.851   0.859  0.851  0.848  0.901      0    2.096     0.203          0
 2  A211108-A211109_1    2     3  DT    0.906    0.851   0.859  0.851  0.848  0.901      0    2.141     0.203          0
 3  A211108-A211109_1    3     4  DT    0.906    0.851   0.859  0.851  0.848  0.901      0    2.109     0.203          0
 4  A211108-A211109_1    4     5  DT    0.906    0.851   0.859  0.851  0.848  0.901      0    2.181     0.203          0



 10%|████████▎                                                                          | 2/20 [01:33<13:54, 46.35s/it]

...

<Figure size 4000x2800 with 0 Axes>

<Figure size 4000x2800 with 0 Axes>

<Figure size 4000x2800 with 0 Axes>

<Figure size 4000x2800 with 0 Axes>

<Figure size 4000x2800 with 0 Axes>

In [15]:
datasets=[ ['./csvs\\A211110.csv', './csvs\\A211112.csv', './csvs\\I211110.csv'],
 ['./csvs\\A211126.csv', './csvs\\A211207.csv'],
 ['./csvs\\I211116.csv', './csvs\\I211117.csv']]


In [16]:
say=0
dataset="./results/vs/"
folder(dataset)
for no, i in enumerate(datasets):
    for ii in i:
        for iii in tqdm(i):
            say+=1
            train=ii
            test=iii
            mixed=False
            step=1
            sayac=1
            dataset="./results/vs/"
            output_csv=f"{dataset}/@{ii[7:-4]}@{iii[7:-4]}.csv"
            dataset=f"{ii[7:-4]}-{iii[7:-4]}"
            target_names=target_name(test)
            ML(train,test,output_csv,feature,step,mixed,dataset[2:-1]+"_"+str(step))   


  0%|                                                                                            | 0/3 [00:00<?, ?it/s]

    Dataset           T    CV  ML      Acc    b_Acc    Prec    Rec     F1    kap    ROC    tra-T    test-T    Al-Time
--  --------------  ---  ----  ----  -----  -------  ------  -----  -----  -----  -----  -------  --------  ---------
 0  11110-A21111_1    0     1  DT    0.906    0.926   0.928  0.926  0.924  0.901      0    3.328     0.25           0
 1  11110-A21111_1    1     2  DT    0.906    0.926   0.928  0.926  0.924  0.901      0    3.078     0.266          0
 2  11110-A21111_1    2     3  DT    0.906    0.926   0.928  0.926  0.924  0.901      0    3.344     0.266          0
 3  11110-A21111_1    3     4  DT    0.906    0.926   0.928  0.926  0.924  0.901      0    3.34      0.297          0
 4  11110-A21111_1    4     5  DT    0.906    0.926   0.928  0.926  0.924  0.901      0    3.379     0.25           0



 33%|████████████████████████████                                                        | 1/3 [00:54<01:49, 54.80s/it]

    Dataset           T    CV  ML      Acc    b_Acc    Prec    Rec     F1    kap    ROC    tra-T    test-T    Al-Time
--  --------------  ---  ----  ----  -----  -------  ------  -----  -----  -----  -----  -------  --------  ---------
 0  11110-A21111_1    0     1  DT    0.823    0.823   0.811  0.823  0.799  0.814      0    3.297     0.203          0
 1  11110-A21111_1    1     2  DT    0.822    0.823   0.819  0.823  0.803  0.814      0    3.422     0.219          0
 2  11110-A21111_1    2     3  DT    0.822    0.822   0.804  0.822  0.795  0.814      0    3.266     0.203          0
 3  11110-A21111_1    3     4  DT    0.822    0.823   0.809  0.823  0.798  0.814      0    3.286     0.219          0
 4  11110-A21111_1    4     5  DT    0.822    0.822   0.804  0.822  0.795  0.814      0    3.297     0.203          0



 67%|████████████████████████████████████████████████████████                            | 2/3 [01:45<00:52, 52.59s/it]

...

<Figure size 4000x2800 with 0 Axes>

<Figure size 4000x2800 with 0 Axes>

<Figure size 4000x2800 with 0 Axes>

<Figure size 4000x2800 with 0 Axes>

<Figure size 4000x2800 with 0 Axes>

In [17]:
datasets=[ 
 ['./csvs\\A211124.csv',
  './csvs\\I211124.csv',
  './csvs\\I211125.csv',
  './csvs\\I211201.csv',
  './csvs\\I211202.csv',
  './csvs\\I211203.csv',
  './csvs\\I211207.csv',
  './csvs\\I211208.csv',
  './csvs\\I211209.csv',
  './csvs\\I211210.csv',
  './csvs\\I211213.csv',
  './csvs\\I211215.csv',
  './csvs\\I211216.csv',
  './csvs\\I211220.csv',
  './csvs\\I211222.csv',
  './csvs\\I211223.csv']]


In [18]:
say=0
dataset="./results/vs/"
folder(dataset)
for no, i in enumerate(datasets):
    for ii in i:
        for iii in tqdm(i):
            say+=1
            train=ii
            test=iii
            mixed=False
            step=1
            sayac=1
            dataset="./results/vs/"
            output_csv=f"{dataset}/@{ii[7:-4]}@{iii[7:-4]}.csv"
            dataset=f"{ii[7:-4]}-{iii[7:-4]}"
            target_names=target_name(test)
            ML(train,test,output_csv,feature,step,mixed,dataset[2:-1]+"_"+str(step))   


  0%|                                                                                           | 0/16 [00:00<?, ?it/s]

    Dataset           T    CV  ML      Acc    b_Acc    Prec    Rec     F1    kap    ROC    tra-T    test-T    Al-Time
--  --------------  ---  ----  ----  -----  -------  ------  -----  -----  -----  -----  -------  --------  ---------
 0  11124-A21112_1    0     1  DT    0.927     0.92   0.927   0.92  0.921  0.924      0    1.391     0.156          0
 1  11124-A21112_1    1     2  DT    0.927     0.92   0.927   0.92  0.921  0.924      0    1.359     0.141          0
 2  11124-A21112_1    2     3  DT    0.927     0.92   0.927   0.92  0.921  0.924      0    1.063     0.141          0
 3  11124-A21112_1    3     4  DT    0.927     0.92   0.927   0.92  0.921  0.924      0    1.375     0.156          0
 4  11124-A21112_1    4     5  DT    0.927     0.92   0.927   0.92  0.921  0.924      0    1.344     0.141          0



  6%|█████▏                                                                             | 1/16 [00:34<08:31, 34.13s/it]

    Dataset           T    CV  ML      Acc    b_Acc    Prec    Rec     F1    kap    ROC    tra-T    test-T    Al-Time
--  --------------  ---  ----  ----  -----  -------  ------  -----  -----  -----  -----  -------  --------  ---------
 0  11124-I21112_1    0     1  DT     0.87    0.855   0.86   0.855  0.843  0.865      0    1.406     0.188          0
 1  11124-I21112_1    1     2  DT     0.87    0.853   0.859  0.853  0.842  0.864      0    1.391     0.188          0
 2  11124-I21112_1    2     3  DT     0.87    0.855   0.86   0.855  0.843  0.865      0    1.375     0.203          0
 3  11124-I21112_1    3     4  DT     0.87    0.855   0.86   0.855  0.843  0.865      0    1.314     0.203          0
 4  11124-I21112_1    4     5  DT     0.87    0.855   0.86   0.855  0.843  0.865      0    1.298     0.188          0



 12%|██████████▍                                                                        | 2/16 [01:12<08:33, 36.66s/it]

...

<Figure size 4000x2800 with 0 Axes>

<Figure size 4000x2800 with 0 Axes>

<Figure size 4000x2800 with 0 Axes>

<Figure size 4000x2800 with 0 Axes>

<Figure size 4000x2800 with 0 Axes>

In [19]:
import winsound
duration = 1000  # milliseconds
freq = 440  # Hz
winsound.Beep(freq, duration)